In [4]:
#!pip install --upgrade tensorflow
#!pip install keras
#!pip install --upgrade pip
#!conda update wrapt --yes
!pip install wrapt

In [1]:
import tensorflow as tf

/home/michael/anaconda3/envs/tensorlow-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/michael/anaconda3/envs/tensorlow-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/michael/anaconda3/envs/tensorlow-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/michael/anaconda

In [2]:
import tensorflow.keras as keras

In [3]:
#importing the necessary libraries
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#models that we are bringing in


Using TensorFlow backend.


In [4]:
tf.__version__

'1.14.0'

In [5]:
#test to check that tensorflow is using gpu
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [5]:
#building the architecture
#lets start with AlexNet
classifier = Sequential()


'''
    template of the networks 
    classifier.add(Convolution2D(, , strides = (), padding = "", activation = ""))
    classifier.add(MaxPooling2D(pool_size = (), strides = (), padding = ))
    classifier.add(BatchNormalization())
'''


#first convolution set
classifier.add(Convolution2D(96, 11, strides = (4,4), padding = 'valid', input_shape = (224,224,3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = "valid"))
classifier.add(BatchNormalization())

#second convolution set
classifier.add(Convolution2D(256, 11, strides = (1,1), padding = "valid", activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = "valid"))
classifier.add(BatchNormalization())

#third convolution set
classifier.add(Convolution2D(384, 3, strides = (1,1), padding = "valid", activation = "relu"))
classifier.add(BatchNormalization())

#fourth convolution set
classifier.add(Convolution2D(384, 3, strides = (1,1), padding = "valid", activation = "relu"))
classifier.add(BatchNormalization())

#fifth convolution set
classifier.add(Convolution2D(256, 3, strides = (1,1), padding = "valid", activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = "valid"))
classifier.add(BatchNormalization())

#flattening step
classifier.add(Flatten())

#dense network
classifier.add(Dense(units = 4096, activation = "relu"))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())

classifier.add(Dense(units = 4096, activation = "relu"))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())

classifier.add(Dense(units = 1000, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())

classifier.add(Dense(units = 10, activation = "softmax"))
classifier.summary()








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 256)         1024      
____________________

In [8]:
classifier.load_weights('../models/best_weights.hdf5')

In [ ]:
#now to  build a custom pipeline
classifier.compile(optimizer = optimizers.SGD(lr=0.001, momentum = 0.9, decay = 0.005), loss="categorical_crossentropy", metrics=['accuracy'])
train_datagen = ImageDataGenerator(rescale= 1./255, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32
base_dir = "../dataset"

training_set = train_datagen.flow_from_directory(base_dir+'/train', target_size=(224,224),batch_size = batch_size, class_mode='categorical')
valid_set = valid_datagen.flow_from_directory(base_dir+'/valid', target_size=(224,224),batch_size = batch_size, class_mode='categorical')

class_dict = training_set.class_indices
li = list(class_dict)

training_num = training_set.samples
valid_num = valid_set.samples

weight_path = '../models/best_weights.hdf5'
checkpoint = ModelCheckpoint(weight_path, monitor='val_acc', verbose=1, save_best_only=True,save_weights_only=True, mode='max')
callbacks_list = [checkpoint]

history = classifier.fit_generator(training_set, steps_per_epoch=training_num//batch_size, validation_data=valid_set, epochs=75, validation_steps=valid_num//batch_size, callbacks=callbacks_list)

Found 18160 images belonging to 10 classes.
Found 4585 images belonging to 10 classes.
Epoch 1/75
567/567 [==============================] - 403s 711ms/step - loss: 0.6262 - acc: 0.7879 - val_loss: 3.2260 - val_acc: 0.4561

Epoch 00001: val_acc improved from -inf to 0.45608, saving model to ../models/best_weights.hdf5
Epoch 2/75
567/567 [==============================] - 385s 679ms/step - loss: 0.5032 - acc: 0.8255 - val_loss: 3.7864 - val_acc: 0.4696

Epoch 00002: val_acc improved from 0.45608 to 0.46958, saving model to ../models/best_weights.hdf5
Epoch 3/75
567/567 [==============================] - 380s 670ms/step - loss: 0.4576 - acc: 0.8416 - val_loss: 3.9486 - val_acc: 0.4579

Epoch 00003: val_acc did not improve from 0.46958
Epoch 4/75
567/567 [==============================] - 447s 789ms/step - loss: 0.4330 - acc: 0.8512 - val_loss: 3.4018 - val_acc: 0.4823

Epoch 00004: val_acc improved from 0.46958 to 0.48232, saving model to ../models/best_weights.hdf5
Epoch 5/75
567/567 [=

In [8]:
#plotting the training progress
sns.set()

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs. acc, color="green", label = "Training Accuracy")
plt.plot(epochs, val_acc, color = 'blue', label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend()
plt.figure()

#loss plot
plt.plot(epochs. loss, color="pink", label = "Training Loss")
plt.plot(epochs, val_loss, color = 'red', label="Validation Loss")
plt.title("Training and Validation Accuracy")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend()
plt.figure()



In [ ]:
#pipeline for predicting new images
image_path = '../dataset/valid/Tomato___Septoria_leaf_spot/f30ddd54-529d-4e4e-bca6-ddad3a68a690___Matt.S_CG 7858.JPG'
new_img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(new_img)